In [2]:
import pandas as pd

df = pd.read_hdf('/content/Netflix_embedding.h5', key='df')





In [3]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
def get_embedding(sentence):
    # Tokenize the sentence
    encoded_input = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        output = model(
            input_ids=encoded_input["input_ids"],
            attention_mask=encoded_input["attention_mask"],
            return_dict=True
        )

    emb = output.last_hidden_state

    mean_emb = emb.mean(dim=1)
    return mean_emb

In [12]:
def find_top_movies(df, input_list, top_n=5):
    input_embedding=get_embedding(input_list)

    df['cosine_similarity'] = df['combined_embedding'].apply(lambda x: cosine_similarity(x.reshape(1, -1), input_embedding)[0][0])

    top_25_movies = df.nlargest(25, 'cosine_similarity')

    top_25_movies_sorted = top_25_movies.sort_values(by='imdb_score', ascending=False)
    final_top_movies = top_25_movies_sorted.head(top_n)

    return final_top_movies['title'].tolist()


In [13]:
input_list = "drama"
top_movies = find_top_movies(df, input_list, top_n=5)

print(top_movies)

['Stories by Rabindranath Tagore', 'House of Cards', 'Move to Heaven', 'El Reemplazante', 'Midnight Diner: Tokyo Stories']
